In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from scipy.stats import ks_2samp
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)

In [2]:
df = pd.read_csv('C:\spring2025\SMAI\Project\output.csv')

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from itertools import combinations

# Copy the dataframe and define the target
threshold = 2
df_copy = df.copy()
df_copy['target'] = ((df_copy['isWicket'] == 1) | (df_copy['run'] <= threshold)).astype(int)

# Define input and target columns
original_input_cols = ['inningNumber', 'oversActual', 'totalRuns', 'totalWickets', 
                       'time_of_day', 'Ground Name', 'Batsman_Name', 'Batsman_Role', 
                       'Batsman_Batting_Style', 'Batsman_Playing_Role']
target_col = 'shotType'  # Or any other target you're currently testing

# Features to ablate
features_to_remove = ['time_of_day', 'oversActual', 'Batsman_Batting_Style']

# Filter good rows and set up X, y
df_good = df_copy[df_copy['target'] == 1]

# Try removing 0 to N features (all combinations)
for i in range(len(features_to_remove) + 1):
    for combo in combinations(features_to_remove, i):
        ablation_tag = ', '.join(combo) if combo else 'None'
        print(f"\n--- Removing: {ablation_tag} ---")

        input_cols = [col for col in original_input_cols if col not in combo]

        X = df_good[input_cols]
        y = df_good[target_col]

        # Optional: One-hot encode if needed (if categorical features aren't already preprocessed)
        X = pd.get_dummies(X)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        clf = RandomForestClassifier(random_state=42)
        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Accuracy: {accuracy:.4f}")



--- Removing: None ---
Accuracy: 0.3792

--- Removing: time_of_day ---
Accuracy: 0.3765

--- Removing: oversActual ---
Accuracy: 0.3695

--- Removing: Batsman_Batting_Style ---
Accuracy: 0.3769

--- Removing: time_of_day, oversActual ---
Accuracy: 0.3552

--- Removing: time_of_day, Batsman_Batting_Style ---
Accuracy: 0.3682

--- Removing: oversActual, Batsman_Batting_Style ---
Accuracy: 0.3648

--- Removing: time_of_day, oversActual, Batsman_Batting_Style ---
Accuracy: 0.3569
